<a href="https://colab.research.google.com/github/aicharya/HomePrice/blob/master/KaggleHomePrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Checkout the repo

In [1]:
!git clone https://github.com/aicharya/HomePrice.git

Cloning into 'HomePrice'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 66 (delta 38), reused 6 (delta 2), pack-reused 0
Unpacking objects: 100% (66/66), done.


Pull the latest files

In [2]:
!cd HomePrice; git pull; ls

Already up to date.
DecisionTree.ipynb		 RandomForest.ipynb	sample_submission.csv
ForwardStepwiseRegression.ipynb  README.md		test.csv
KaggleHomePrice.ipynb		 RegressionTrees.ipynb	train.csv
LassoRegression.ipynb		 RidgeRegression.ipynb


In [3]:
import os
print(os.listdir("HomePrice"))

['.git', 'README.md', 'sample_submission.csv', 'LassoRegression.ipynb', 'test.csv', 'ForwardStepwiseRegression.ipynb', 'DecisionTree.ipynb', 'train.csv', 'RandomForest.ipynb', 'RegressionTrees.ipynb', 'KaggleHomePrice.ipynb', 'RidgeRegression.ipynb']


Load Data

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X_full = pd.read_csv('HomePrice/train.csv', index_col='Id')
X_test_full = pd.read_csv('HomePrice/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll use only numerical predictors
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

Model scoring with MAE

In [0]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor


# Function for comparing different models
def score_model(model, X_t=X_train, X_v=X_valid, y_t=y_train, y_v=y_valid):
    model.fit(X_t, y_t)
    preds = model.predict(X_v)
    return mean_absolute_error(y_v, preds)
  

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)


Find missing data values

In [12]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())

print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


Data wrangling - Approach 1 

Drop missing data

In [0]:
cols_missing_data = [col for col in X_train.columns if X_train[col].isnull().any()]

cleaned_X_train = X_train.drop(cols_missing_data, axis=1)
cleaned_X_valid = X_valid.drop(cols_missing_data, axis=1)

shape after cleaning

In [15]:
# Shape of training data (num_rows, num_columns)
print(cleaned_X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (cleaned_X_train.isnull().sum())

print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 33)
Series([], dtype: int64)


MAE after dropping columns with missing data

In [18]:
print("MAE (Drop columns with missing values):")
print(score_dataset(cleaned_X_train, cleaned_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17837.82570776256


Forward Stepwise Regression

Ridge Regression

Lasso Regression

Regression Tree

The Optimal Model